In [1]:
import sqlite3
import pandas as pd

### sqlite3 db to Dataframe

In [2]:
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()

In [3]:
df_store = pd.read_sql_query("SELECT * from api_store", con)
df_review = pd.read_sql_query("SELECT * from api_review", con)

In [4]:
df_store.head()

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차,0,0
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페,0,0
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍,0,0
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제,0,0
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트,0,0


In [5]:
df_review.head()

,id,total_score,content,reg_time,store_id,user_id
0,1,5.0,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!,2020-04-06 01:19:01.604070,15,68632
1,2,5.0,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천,2020-04-06 01:19:01.605069,18,389728
2,3,4.0,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...,2020-04-06 01:19:01.605069,19,68716
3,4,2.0,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...",2020-04-06 01:19:01.605069,37,774353
4,5,3.0,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...,2020-04-06 01:19:01.605069,38,115682


In [6]:
con.close()

### surprise
#### https://medium.com/sfu-cspmp/recommendation-systems-user-based-collaborative-filtering-using-n-nearest-neighbors-bf7361dc24e0

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [8]:
stores = df_store[['id','store_name','category']]
reviews = df_review[['user_id','store_id','total_score']]
reviews.rename(columns={'total_score':'rating'}, inplace=True)
stores.rename(columns={'id':'store_id'},inplace=True)

C:\Users\multicampus\anaconda3\envs\pjt\lib\site-packages\pandas\core\frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [9]:
Mean = reviews.groupby(by="user_id",as_index=False)['rating'].mean()
Rating_avg = pd.merge(reviews,Mean,on='user_id')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

,user_id,store_id,rating_x,rating_y,adg_rating
0,68632,15,5.0,4.842975,0.157025
1,68632,1216,5.0,4.842975,0.157025
2,68632,8756,5.0,4.842975,0.157025
3,68632,9460,5.0,4.842975,0.157025
4,68632,13745,5.0,4.842975,0.157025


In [10]:
final = pd.pivot_table(Rating_avg,values='adg_rating',index='user_id',columns='store_id')
final.head()

store_id,15,18,19,37,38,49,50,53,56,58,...,360481,360482,360486,360489,360492,360493,360499,360505,360514,360515
user_id,,,,,,,,,,,,,,,,,,,,,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Replacing NaN by Movie Average
final_store = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [12]:
# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

user_id,7,15,23,62,64,74,105,137,147,156,...,949436,949554,949596,949670,949944,949946,949951,950224,950225,950331
user_id,,,,,,,,,,,,,,,,,,,,,
7,0.000000e+00,0.0,1.735714e-27,0.0,0.0,0.0,-1.456467e-27,0.0,4.336490e-28,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.675468e-28,0.0,0.0
15,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
23,1.735714e-27,0.0,0.000000e+00,0.0,0.0,0.0,-4.206993e-28,0.0,1.237613e-28,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.510132e-28,0.0,0.0
62,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
64,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0


In [13]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_store)
np.fill_diagonal(cosine, 0 )
similarity_with_store = pd.DataFrame(cosine,index=final_store.index)
similarity_with_store.columns=final_user.index
similarity_with_store.head()

user_id,7,15,23,62,64,74,105,137,147,156,...,949436,949554,949596,949670,949944,949946,949951,950224,950225,950331
user_id,,,,,,,,,,,,,,,,,,,,,
7,0.000000,0.999990,0.999984,0.999994,0.999994,0.999994,0.999947,0.999992,0.999456,0.999988,...,0.999994,0.999990,0.999993,0.999932,0.999994,0.999994,0.999994,0.999778,0.999994,0.999991
15,0.999990,0.000000,0.999985,0.999996,0.999996,0.999996,0.999949,0.999993,0.999458,0.999990,...,0.999996,0.999991,0.999994,0.999934,0.999996,0.999996,0.999996,0.999780,0.999996,0.999993
23,0.999984,0.999985,0.000000,0.999990,0.999990,0.999990,0.999943,0.999987,0.999451,0.999983,...,0.999990,0.999985,0.999988,0.999928,0.999990,0.999990,0.999990,0.999774,0.999990,0.999986
62,0.999994,0.999996,0.999990,0.000000,1.000000,1.000000,0.999953,0.999997,0.999462,0.999994,...,1.000000,0.999996,0.999998,0.999938,1.000000,1.000000,1.000000,0.999784,1.000000,0.999997
64,0.999994,0.999996,0.999990,1.000000,0.000000,1.000000,0.999953,0.999997,0.999462,0.999994,...,1.000000,0.999996,0.999998,0.999938,1.000000,1.000000,1.000000,0.999784,1.000000,0.999997


In [ ]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
def get_user_similar_stores( user1, user2 ):
    common_stores = Rating_avg[Rating_avg.user_id == user1].merge(
    Rating_avg[Rating_avg.user_id == user2],
    on = "store_id",
    how = "inner" )
    return common_stores.merge( stores, on = 'store_id' )

In [ ]:
a = get_user_similar_movies(370,86309)
a = a.loc[ : , ['rating_x_x','rating_x_y','title']]
a.head()

In [ ]:
#swapping columns
raw=df[['user_id','store_id','total_score']] 
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(1,5)) 
#into surprise:
data = surprise.Dataset.load_from_df(raw,reader)

In [31]:
class ProbabilisticMatrixFactorization(surprise.AlgoBase):
# Randomly initializes two Matrices, Stochastic Gradient Descent to be able to optimize the best factorization for ratings.
    def __init__(self,learning_rate,num_epochs,num_factors):
       # super(surprise.AlgoBase)
        self.alpha = learning_rate #learning rate for Stochastic Gradient Descent
        self.num_epochs = num_epochs
        self.num_factors = num_factors
    def fit(self,train):
        #randomly initialize user/item factors from a Gaussian
        P = np.random.normal(0,.1,(train.n_users,self.num_factors))
        Q = np.random.normal(0,.1,(train.n_items,self.num_factors))
        #print('fit')

        for epoch in range(self.num_epochs):
            for u,i,r_ui in train.all_ratings():
                residual = r_ui - np.dot(P[u],Q[i])
                temp = P[u,:] # we want to update them at the same time, so we make a temporary variable. 
                P[u,:] +=  self.alpha * residual * Q[i]
                Q[i,:] +=  self.alpha * residual * temp 

                
        self.P = P
        self.Q = Q

        self.trainset = train
    
    
    def estimate(self,u,i):
        #returns estimated rating for user u and item i. Prerequisite: Algorithm must be fit to training set.
        #check to see if u and i are in the train set:
        #print('gahh')

        if self.trainset.knows_user(u) and self.trainset.knows_item(i):
            #print(u,i, '\n','yep:', self.P[u],self.Q[i])
            #return scalar product of P[u] and Q[i]
            nanCheck = np.dot(self.P[u],self.Q[i])
            
            if np.isnan(nanCheck):
                return self.trainset.global_mean
            else:
                return np.dot(self.P[u,:],self.Q[i,:])
        else:# if its not known we'll return the general average. 
           # print('global mean')
            return self.trainset.global_mean

In [32]:
Alg1 = ProbabilisticMatrixFactorization(learning_rate=0.05,num_epochs=4,num_factors=10)
data1 = data.build_full_trainset()
Alg1.fit(data1)
print(raw.user_id.iloc[4],raw.store_id.iloc[4])
Alg1.estimate(raw.user_id.iloc[4],raw.store_id.iloc[4])

C:\Users\multicampus\anaconda3\envs\pjt\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in multiply
C:\Users\multicampus\anaconda3\envs\pjt\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in multiply


115682 38


3.8466049585330095

In [33]:
gs = surprise.model_selection.GridSearchCV(ProbabilisticMatrixFactorization, param_grid={'learning_rate':[0.005,0.01],
                                                                            'num_epochs':[5,10],
                                                                            'num_factors':[10,20]},measures=['rmse', 'mae'], cv=2)
gs.fit(data)

In [34]:
print('rsme: ',gs.best_score['rmse'],'mae: ',gs.best_score['mae'])
best_params = gs.best_params['rmse']
print('rsme: ',gs.best_params['rmse'],'mae: ',gs.best_params['mae'])

rsme:  2.2151622698170197 mae:  1.7982508481108828
rsme:  {'learning_rate': 0.01, 'num_epochs': 10, 'num_factors': 20} mae:  {'learning_rate': 0.01, 'num_epochs': 10, 'num_factors': 20}


In [35]:
bestVersion = ProbabilisticMatrixFactorization(learning_rate=best_params['learning_rate'],num_epochs=best_params['num_epochs'],num_factors=best_params['num_factors'])

In [36]:
#we can use k-fold cross validation to evaluate the best model. 
kSplit = surprise.model_selection.KFold(n_splits=10,shuffle=True)
for train,test in kSplit.split(data):
    bestVersion.fit(train)
    prediction = bestVersion.test(test)
    surprise.accuracy.rmse(prediction,verbose=True)

RMSE: 2.3195
RMSE: 2.3201
RMSE: 2.3191
RMSE: 2.3084
RMSE: 2.3284
RMSE: 2.3073
RMSE: 2.3121
RMSE: 2.2980
RMSE: 2.3339
RMSE: 2.3106


#### cross_validate parameters 
+ algo ( AlgoBase) – 평가할 알고리즘입니다.
+ data ( Dataset) – 알고리즘을 평가할 데이터 세트입니다.
측정 값 ( 문자열 목록 ) – 계산할 성능 측정 값입니다. 허용되는 이름은 accuracy모듈에 정의 된 기능 이름 입니다. 기본값은 입니다.['rmse', 'mae']
+ cv (cross-validation iterator, int 또는 None) – data매개 변수가 분할되는 방법 (트레인 셋 및 테스트 세트가 정의되는 방법)을 결정합니다. int가 전달되면 KFold적절한 n_splits매개 변수 와 함께 사용됩니다 . 경우 None, KFold사용됩니다 n_splits=5.
+ return_train_measures ( bool ) – 기차 세트에서 성능 측정 값을 계산할지 여부입니다. 기본값은 False입니다.
n_jobs ( int ) –
병렬로 평가 된 최대 접기 수입니다.

인 경우 -1모든 CPU가 사용됩니다.
1주어진 경우 병렬 컴퓨팅 코드가 전혀 사용되지 않으므로 디버깅에 유용합니다.
내용은 n_jobs아래에 -1, 사용된다. 예를 들어 하나의 CPU를 제외한 모든 CPU가 사용됩니다.(n_cpus + n_jobs + 1)n_jobs = -2
기본값은 1입니다.

+ pre_dispatch ( int 또는 string ) –
병렬 실행 중에 디스패치되는 작업 수를 제어합니다. 이 수를 줄이면 CPU가 처리 할 수있는 것보다 더 많은 작업이 디스패치 될 때 메모리 소비가 급격히 증가하는 것을 피할 수 있습니다. 이 매개 변수는 다음과 같습니다.

None이 경우 모든 작업이 즉시 생성되어 생성됩니다. 작업의 주문형 생성으로 인한 지연을 피하기 위해 가볍고 빠르게 실행되는 작업에 사용하십시오.
생성 된 정확한 총 작업 수를 제공하는 정수입니다.
에서와 같이의 함수로 표현식을 제공하는 문자열 n_jobs입니다 '2*n_jobs'.
기본값은 '2*n_jobs'입니다.

+ verbose ( int ) – True각 분할에 대한 정확도 측정 값이 인쇄되고 훈련 및 테스트 시간 모든 분할에 대한 평균 및 표준 편차도보고됩니다. 기본값은 False아무것도 인쇄되지 않습니다.

In [ ]:
https://www.fun-coding.org/recommend_basic7.html

In [66]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import dump
import os

#swapping columns
raw=df[['user_id','store_id','total_score']] 
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(0,5)) 
#into surprise:
dataframe = surprise.Dataset.load_from_df(raw,reader)
trainset = dataframe.build_full_trainset()
# Use the famous SVD algorithm.
algo = SVD()
algo.fit(trainset)

predictions = algo.test(trainset.build_testset())

# Dump algorithm and reload it.
file_name = os.path.expanduser('dump_file')
dump.dump(file_name, algo=algo)
_, loaded_algo = dump.load(file_name)

# We now ensure that the algo is still the same by checking the predictions.
predictions_loaded_algo = loaded_algo.test(trainset.build_testset())
assert predictions == predictions_loaded_algo
print('Predictions are the same')

Predictions are the same


In [69]:
# 정확도 계산
from surprise import Dataset
from surprise import SVD
from surprise import accuracy

#swapping columns
raw=df[['user_id','store_id','total_score']] 
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(0,5)) 
#into surprise:
dataframe = surprise.Dataset.load_from_df(raw,reader)
algo = SVD()

trainset = data.build_full_trainset()
algo.fit(trainset)

testset = trainset.build_testset()
predictions = algo.test(testset)

accuracy.rmse(predictions)

RMSE: 0.6744


0.6744237205825014

In [71]:
# 모델 최적화
import random

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import GridSearchCV


# Load the full dataset.
#swapping columns
raw=df[['user_id','store_id','total_score']] 
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(0,5)) 
#into surprise:
dataframe = surprise.Dataset.load_from_df(raw,reader)

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(dataframe)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.0482671327202466
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [74]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [72]:
# First train an SVD algorithm on the movielens dataset.
raw=df[['user_id','store_id','total_score']] 
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(0,5)) 
#into surprise:
dataframe = surprise.Dataset.load_from_df(raw,reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

[Prediction(uid='196', iid='302', r_ui=3.52986, est=3.883043649424666, details={'was_impossible': False}),
 Prediction(uid='196', iid='377', r_ui=3.52986, est=2.752335091389584, details={'was_impossible': False}),
 Prediction(uid='196', iid='51', r_ui=3.52986, est=3.3187020243376866, details={'was_impossible': False}),
 Prediction(uid='196', iid='346', r_ui=3.52986, est=3.4479993814693195, details={'was_impossible': False}),
 Prediction(uid='196', iid='474', r_ui=3.52986, est=4.501962498243956, details={'was_impossible': False}),
 Prediction(uid='196', iid='265', r_ui=3.52986, est=4.044989069864434, details={'was_impossible': False}),
 Prediction(uid='196', iid='465', r_ui=3.52986, est=3.752545552157244, details={'was_impossible': False}),
 Prediction(uid='196', iid='451', r_ui=3.52986, est=3.3177431303682825, details={'was_impossible': False}),
 Prediction(uid='196', iid='86', r_ui=3.52986, est=3.8433447178605933, details={'was_impossible': False}),
 Prediction(uid='196', iid='1014', 

In [75]:
top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

196 ['483', '180', '313', '64', '178', '603', '513', '474', '480', '357']
186 ['174', '408', '496', '318', '172', '205', '50', '498', '186', '923']
22 ['169', '408', '12', '183', '483', '191', '114', '652', '963', '302']
244 ['127', '131', '14', '483', '12', '515', '514', '482', '606', '644']
166 ['511', '169', '513', '408', '172', '114', '318', '185', '246', '48']
298 ['64', '169', '191', '302', '12', '480', '923', '251', '408', '493']
115 ['169', '484', '179', '483', '474', '168', '134', '64', '156', '276']
253 ['498', '172', '114', '178', '28', '603', '205', '694', '174', '269']
305 ['603', '647', '488', '519', '213', '657', '516', '114', '137', '694']
6 ['428', '603', '657', '179', '647', '611', '963', '654', '656', '190']
62 ['169', '408', '963', '175', '519', '657', '223', '178', '480', '647']
286 ['705', '9', '521', '320', '318', '651', '59', '12', '659', '165']
200 ['272', '64', '12', '651', '316', '251', '520', '302', '479', '427']
210 ['318', '64', '496', '169', '272', '480',

715 ['114', '357', '190', '178', '483', '515', '169', '272', '191', '530']
713 ['318', '483', '408', '98', '64', '169', '474', '199', '12', '60']
716 ['169', '320', '114', '408', '166', '923', '59', '313', '1131', '198']
681 ['318', '313', '357', '98', '178', '191', '272', '114', '483', '515']
678 ['64', '98', '483', '313', '408', '318', '357', '427', '513', '169']
719 ['12', '480', '178', '651', '272', '515', '100', '603', '114', '173']
702 ['169', '48', '59', '89', '100', '174', '190', '114', '318', '963']
721 ['408', '96', '169', '178', '480', '195', '144', '483', '313', '1019']
714 ['408', '169', '64', '313', '511', '483', '512', '603', '963', '186']
717 ['316', '479', '251', '427', '185', '318', '169', '114', '480', '657']
718 ['483', '318', '603', '64', '408', '12', '313', '178', '114', '169']
696 ['357', '192', '318', '515', '408', '169', '483', '191', '127', '251']
722 ['50', '12', '169', '174', '480', '483', '178', '408', '172', '272']
724 ['64', '169', '496', '318', '480', '1

In [77]:
df_review[df_review['user_id']==943]

,id,total_score,content,reg_time,store_id,user_id


,id,total_score,store_id,user_id
count,91398.000000,91398.000000,91398.000000,91398.000000
mean,45699.500000,3.846605,184286.576927,324560.215782
std,26384.474289,1.109838,101038.387563,255908.402479
min,1.000000,0.000000,15.000000,7.000000
25%,22850.250000,3.000000,100178.000000,127124.000000
50%,45699.500000,4.000000,187034.500000,255093.000000
75%,68548.750000,5.000000,269498.000000,469245.000000
max,91398.000000,5.000000,360515.000000,950331.000000


#### https://lsjsj92.tistory.com/570?category=853217

In [43]:
df_user_store_rating = df_review.pivot(
index='user_id', columns='store_id', values='total_score')

In [44]:
df_user_store_rating.head()

store_id,15,18,19,37,38,49,50,53,56,58,...,360481,360482,360486,360489,360492,360493,360499,360505,360514,360515
user_id,,,,,,,,,,,,,,,,,,,,,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
matrix = df_user_store_rating.as_matrix()
user_rating_mean = np.mean(matrix, axis=1)
matrix_user_mean = matrix - user_rating_mean.reshape(-1,1)

C:\Users\multicampus\anaconda3\envs\pjt\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [46]:
pd.DataFrame(matrix_user_mean, columns= df_user_store_rating.columns).head()

store_id,15,18,19,37,38,49,50,53,56,58,...,360481,360482,360486,360489,360492,360493,360499,360505,360514,360515
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
from scipy.sparse.linalg import svds, eigs

In [50]:
U, sigma, Yt = svds(matrix_user_mean, k=12)
print(U.shape)
print(sigma.shape)
print(Yt.shape)

(18992, 12)
(12,)
(12, 45700)


C:\Users\multicampus\anaconda3\envs\pjt\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1843: RuntimeWarning: invalid value encountered in greater
  above_cutoff = (eigvals > cutoff)


In [51]:
sigma = np.diag(sigma)
sigma.shape

(12, 12)

In [52]:
svd_user_predicted_ratings = np.dot(np.dot(U,sigma), Yt) + user_rating_mean.reshape(-1,1)

In [53]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns=df_user_store_rating.columns)
df_svd_preds.head()

store_id,15,18,19,37,38,49,50,53,56,58,...,360481,360482,360486,360489,360492,360493,360499,360505,360514,360515
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
def recommend_store(df_svd_preds, user_id, ori_store_df,ori_review_df, num_recommendations=5):
    user_row_number = user_id
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    user_data = ori_review_df[ori_review_df.user_id==user_row_number]
    user_history = pd.merge(user_data,ori_store_df, left_on="store_id", right_on ="id").sort_values(['total_score'], ascending=False)
    
    recommendations = ori_store_df[~ori_store_df['id'].isin(user_history['store_id'])]
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on="id")
    recommendations = recommendations.rename(columns={user_row_number:'Predictions'}).sort_values('Predictions',ascending=False)
    return user_history, recommendations

In [61]:
already_rated , predictions = recommend_store(df_svd_preds,469245, df_store,df_review,10)

IndexError: single positional indexer is out-of-bounds

In [ ]:
already_rated.head()

In [56]:
df_review['user_id'].value_counts()

469245    461
243883    389
328775    380
391794    362
74999     352
         ... 
867388      1
461882      1
70713       1
44132       1
155636      1
Name: user_id, Length: 18992, dtype: int64